In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/FireStatLA/UCLA Heat Days - Daily Incident Summary.csv', parse_dates=['Incident Date'])

df.head()

,Incident Date,FSDistrict,IncidentCategory,IncidentCount
0,2018-01-01,1,AUTOMOBILE,1
1,2018-01-01,1,BURNS,1
2,2018-01-01,1,CHEST DISCOMFORT/HEART PROBLEM,1
3,2018-01-01,1,DIABETIC,1
4,2018-01-01,1,FALL,2


In [3]:
df.tail()

,Incident Date,FSDistrict,IncidentCategory,IncidentCount
1475661,2022-12-31,109,TRAFFIC,1
1475662,2022-12-31,112,ASSAULT,1
1475663,2022-12-31,112,CHEST DISCOMFORT/HEART PROBLEM,1
1475664,2022-12-31,112,FALL,3
1475665,2022-12-31,112,O.D./POISONING,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1475666 entries, 0 to 1475665
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Incident Date     1475666 non-null  datetime64[ns]
 1   FSDistrict        1475666 non-null  int64         
 2   IncidentCategory  1474711 non-null  object        
 3   IncidentCount     1475666 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 45.0+ MB


In [5]:
df.IncidentCategory.unique()

array(['AUTOMOBILE', 'BURNS', 'CHEST DISCOMFORT/HEART PROBLEM',
       'DIABETIC', 'FALL', 'INTERFACILITY TRANSFER', 'O.D./POISONING',
       'RUBBISH', 'TRAFFIC', 'TREE', 'UNCONSCIOUS/FAINTING',
       'UNKNOWN PROBLEM', 'PEDIATRICS', 'SICK', 'STRUCTURE',
       'WATER-RELATED', 'ABDOMINAL', 'ASSAULT', 'AUTOMATIC ALARM',
       'CHEMICAL', 'SEIZURE', 'STABBING', 'STROKE', 'TOXIC',
       'PSYCH/SA/EMOTIONAL', 'BREATHING DIFFICULTY', 'INJURY', 'ELEVATOR',
       'BLEEDING (NON-TRAUMATIC)', 'SMOKE DETECTOR', 'GRASS',
       'ALLERGY/ENVENOMATION', 'CARDIAC/RESP. ARREST', 'CHOKING',
       'ANIMAL BITE', 'LOCKOUT', 'ELECTRICAL', 'BELLS RINGING',
       'SHOOTING', 'OB/GYN', 'SMOKE', 'STRUCTURE-AA', 'VEHICULAR', 'POLE',
       'HEADACHE', 'OVEN', 'BRUSH', 'H/C EXPOSURE', 'WATER FLOW',
       'BACK PAIN', 'HELP', 'TRANSFORMER', 'JUMPER', 'ELECTROCUTION',
       'AIRCRAFT', 'TOXIC EXPOSURE', 'INFECTIOUS DISEASE', 'CLIFF/HIKER',
       'DEBRIS', 'BOAT', 'BURN', 'BRUSHAA', 'TAR', 'METRO EL', 

In [6]:
ems_codes = pd.read_csv('data/FireStatLA/EMS Codes.csv')
ems_codes = ems_codes.melt() # creates 'long' dataframe of all ems values
ems_codes.dropna(inplace=True) #get rid of extra na values
ems_codes.rename(columns={'value': 'EMS', 'variable': 'severity'}, inplace=True) # renames columns to more intuitive names

ems_codes.head()

,severity,EMS
0,AHR,ABDOMINAL
1,AHR,ALLERGY/ENVENOMATION
2,AHR,ANIMAL BITE
3,AHR,ASSAULT
4,AHR,BLEEDING(NON-TRAUMATIC)


In [7]:
ems_df = df[df['IncidentCategory'].isin(ems_codes['EMS'])]
ems_df.head()

,Incident Date,FSDistrict,IncidentCategory,IncidentCount
1,2018-01-01,1,BURNS,1
2,2018-01-01,1,CHEST DISCOMFORT/HEART PROBLEM,1
3,2018-01-01,1,DIABETIC,1
4,2018-01-01,1,FALL,2
5,2018-01-01,1,INTERFACILITY TRANSFER,1


I think the next step here will be to group by date and FS district with the aggregation being a sum of Incident Count, since it seems like that is the volume for the day in that district.

In [10]:
print('The number of EMS records is: {}'.format(len(ems_df)))
print('The number of individual EMS calls is: {}'.format(ems_df['IncidentCount'].sum()))

The number of EMS records is: 1095001
The number of individual EMS calls is: 1679765
